placeholder

In [1]:
import xarray as xr
import pandas as pd
import numpy as np
import dask.array as da
from dask.distributed import Client
import xskillscore as xs
%load_ext blackcellmagic

In [2]:
client = Client()
client

Client Scheduler: tcp://127.0.0.1:44685 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 3 Cores: 6 Memory: 16.73 GB


Due to the success of your previous forecast (and choosing which metric to optimize using xskillscore!) the company you work for has expanded...

In [3]:
stores = np.arange(400)
skus = np.arange(3000)

In [4]:
dates = pd.date_range("1/1/2020", "12/31/2020", freq="D")

In [5]:
data = da.random.randint(10, size=(len(dates), len(stores), len(skus)))
data

dask.array<randint, shape=(366, 400, 3000), dtype=int64, chunksize=(183, 200, 250), chunktype=numpy.ndarray>

In [6]:
y = xr.DataArray(data, coords=[dates, stores, skus], dims=["DATE", "STORE", "SKU"])
y

<xarray.DataArray 'randint-006663fcb006c7ff42986c6061271f59' (DATE: 366, STORE: 400, SKU: 3000)>
dask.array<randint, shape=(366, 400, 3000), dtype=int64, chunksize=(183, 200, 250), chunktype=numpy.ndarray>
Coordinates:
  * DATE     (DATE) datetime64[ns] 2020-01-01 2020-01-02 ... 2020-12-31
  * STORE    (STORE) int64 0 1 2 3 4 5 6 7 8 ... 392 393 394 395 396 397 398 399
  * SKU      (SKU) int64 0 1 2 3 4 5 6 7 ... 2993 2994 2995 2996 2997 2998 2999

In [7]:
noise = da.random.uniform(-1, 1, size=(len(dates), len(stores), len(skus)))
yhat = y + (y * noise)
yhat

<xarray.DataArray (DATE: 366, STORE: 400, SKU: 3000)>
dask.array<add, shape=(366, 400, 3000), dtype=float64, chunksize=(183, 200, 250), chunktype=numpy.ndarray>
Coordinates:
  * DATE     (DATE) datetime64[ns] 2020-01-01 2020-01-02 ... 2020-12-31
  * STORE    (STORE) int64 0 1 2 3 4 5 6 7 8 ... 392 393 394 395 396 397 398 399
  * SKU      (SKU) int64 0 1 2 3 4 5 6 7 ... 2993 2994 2995 2996 2997 2998 2999

Set chunk to one on the dimension to reduce over...

In [8]:
xs.rmse(y.chunk({'DATE': -1}), yhat.chunk({'DATE': -1}), 'DATE').compute().values

array([[3.08313121, 3.23454991, 3.02335321, ..., 3.02154803, 3.0891383 ,
        3.08582152],
       [3.17561689, 3.0462068 , 3.07660178, ..., 2.9155723 , 2.89765487,
        2.82834411],
       [3.15871151, 3.10288995, 3.21174762, ..., 2.99512422, 3.05851634,
        3.07042754],
       ...,
       [2.78349084, 2.93475867, 3.27630567, ..., 3.07162753, 2.90819859,
        3.28357181],
       [3.01603701, 3.2932717 , 3.39003442, ..., 3.12567319, 3.00259034,
        3.00044338],
       [3.1341985 , 3.16609289, 2.89506904, ..., 2.9268608 , 3.20011582,
        3.08316171]])

Next year you are expected to grow again due to some great forecasts etc...

In [13]:
stores = np.arange(4000)
skus = np.arange(3000)
dates = pd.date_range("1/1/2021", "12/31/2021", freq="D")

data = da.random.randint(10, size=(len(dates), len(stores), len(skus)))
data

,Array,Chunk
Bytes,35.04 GB,127.50 MB
Shape,"(365, 4000, 3000)","(255, 250, 250)"
Count,384 Tasks,384 Chunks
Type,int64,numpy.ndarray


In [14]:
y = xr.DataArray(data, coords=[dates, stores, skus], dims=["DATE", "STORE", "SKU"])

noise = da.random.uniform(-1, 1, size=(len(dates), len(stores), len(skus)))
yhat = y + (y * noise)
yhat

,Array,Chunk
Bytes,35.04 GB,127.50 MB
Shape,"(365, 4000, 3000)","(255, 250, 250)"
Count,1536 Tasks,384 Chunks
Type,float64,numpy.ndarray


Go enjoy your lunch...

In [15]:
xs.rmse(y.chunk({'DATE': -1}), yhat.chunk({'DATE': -1}), 'DATE').compute().values

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


KeyboardInterrupt: 